# **Programming Assessment \#4**

Names: Enriquez, Manolo L.  Narvaez, Jose Wilfredo S.

More information on the assessment is found in our Canvas course. Link: https://dlsu.instructure.com/courses/93383/assignments/739602

# **Load Data**

*While you don't have to separate your code into blocks, it might be easier if you separated loading your data from actually implementation of your code. Consider placing all loading of data into the code block below.*

In [ ]:
import nltk
from collections import Counter
nltk.download('gutenberg')

# Language Model
all_words = Counter()
for filename in nltk.corpus.gutenberg.fileids():
    words = [word.lower() for word in nltk.corpus.gutenberg.words(filename)]
    all_words.update(words)

total_tokens = sum(all_words.values())
total_words = len(all_words)

# Loading error model text file into dictionary
error_model = {}
file1 = open('count_1edit.txt', 'r')
Lines = file1.readlines()

for line in Lines:
    key = line.split('\t')[0]
    value = int(line.split('\t')[1])
    error_model[key] = value



# **Noisy Channel Model Implementation**

*Again, you don't have to follow this directly, but consider placing your implementation of the model in the code block below. And while we discussed the general approach in class, kindly describe how you decided to implement the spell correction model. Include any modifications your group made as well. This might be a good spot to place part of your write up.*

In [ ]:
word = input("Input: ")

one_edit_list = []
output = {}

if word not in all_words:
    # Look for words with one edit distance in language model
    for key in all_words:
        edit_distance = nltk.edit_distance(word, key, substitution_cost=1, transpositions=True)
        if edit_distance == 1:
            one_edit_list.append(key)
     
    # iterate through each word with edit distance of 1
    for candidate in one_edit_list:
        
        # calculate p(word) by getting word occurence count divided 
        # by total number of words in corpus
        p_word = all_words[candidate] / total_words
        
        # getting backtrace for edit distance alignment
        backtrace = nltk.edit_distance_align(word, candidate, substitution_cost=1)
        
        s1 = word
        s2 = candidate
        prev_idx_1 = ''
        prev_idx_2 = ''
        
        # traverse tuple in reverse order to detect the errors
        for pair in reversed(backtrace):
            error_letter = ''
            correct_letter = ''
            error =''
            s1_idx = pair[0]-1
            s2_idx = pair[1]-1
            
            if s1_idx >= 0  and s2_idx >= 0:
                
                
                if s1_idx == s2_idx:
                    
                    # if the current tuples are equal, check if the previous tuple pair if they are different
                    # checks for deletion
                    if prev_idx_1 != s1_idx and prev_idx_2 == s2_idx:
                        error_letter = s1[s1_idx] + s1[s1_idx+1]
                        correct_letter = s2[s2_idx]
                        break
                        
                    # checks for insertion
                    elif prev_idx_1 == s1_idx and prev_idx_2 != s2_idx:
                        error_letter = s1[s1_idx-1]
                        correct_letter = s2[s2_idx-1] + s2[s2_idx] 
                        break
                        
                    # if the tuple is equal but index character of input and candidate is different    
                    # checks for substitution
                    elif s1[s1_idx] != s2[s2_idx]:
                        error_letter = s1[s1_idx]
                        correct_letter = s2[s2_idx]
                        break 

                prev_idx_1 = s1_idx 
                prev_idx_2 = s2_idx 
             
        # create x|w / error
        error = error_letter + '|' + correct_letter
        
        # if the error exists in the error model, calculate noisy channel probability
        if error in error_model:
            ctr = 0
            for words in all_words:
                if correct_letter in words:
                    ctr += 1
                    
            # *P(x|w)P(w) 
            output[candidate] = (error_model[error] / ctr) * p_word
            
    # sorting output dictionary by the value
    print('Output: ', end ="")
    output_list =  sorted(output.items(), key=lambda x:x[1])

    # prints out all the candidates for spelling checking and their probability 
    for i in reversed(output_list):
        print(i[0] + " (" + str(i[1]) + ")", end = ", ")
else:
    print("Output: No error")


# **Your Relfection / Takeaway / Analysis**

*Kindly place the rest of your write up. More information is found in the Canvas write up.*